## Migration Testing - Leonardo

#### Read in Currents feed

In [17]:
from dotenv import load_dotenv
from currentsapi import CurrentsAPI
import os
import json
import requests
import time
import openai
import random

In [18]:
load_dotenv()
currents_api_key = os.environ['currents_api_key']
leonardo_api_key = os.environ['leonardo_key']
openai.api_key = os.environ['openai_key']

In [19]:
api = CurrentsAPI(api_key=currents_api_key)

In [20]:
news_feed = api.search(country="", language="en", category="world")

#### Process into GPT prompt

In [21]:
titles = [x['title'] for x in news_feed['news']]

In [33]:
selected_snippet = random.choice(titles)

In [34]:
selected_snippet

'Some Republicans Were Willing to Compromise on Abortion Ban Exceptions. Activists Made Sure They Didn’t.'

In [35]:
prompt_template = '''
Using the following snippet encased in ```, generate an image prompt of an imaginery character  that is relevant to the snippet, be extravagent and detailed in the description of this character including the type of creature (if applicable) , race, age group, ethnicity and appearance. The character need not seem like someone that exists in the world currently.
Also come up with a creative backstory in around 50 words on this character's origin story and his/her/its main superpower, and include some elements of the original snippet in the backstory.

```Bangladesh Arrest Thousands in 'Violent' Crackdown: HRW```
'''  # noqa: E501

response_template = '''
Imaginary Character: Oracle Lumineer

Description:
Oracle Lumineer, an ethereal being, appears as an ageless cosmic seer with radiant iridescent skin. Their eyes, gleaming with interstellar wisdom, reflect the struggles of oppressed souls. Adorned in celestial robes, they embody the resilience of hope in the face of darkness.

Backstory:
Originating from a cosmic realm, Oracle Lumineer descended to Earth as a response to cries for justice. Infused with the cosmic energy of empathy, they can traverse time and space, seeking out injustice to intervene and inspire change.

Main Superpower:
Cosmic Empathy - Oracle Lumineer possesses the ability to empathize with the collective suffering of oppressed individuals. Drawing on cosmic energies, they channel empathy to influence hearts and minds, fostering unity and inspiring resistance against systemic injustice.'''  # noqa: E501


In [36]:
completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                'role': 'system',
                'content': 'You are a creative character designer'
            },
            {
                'role': 'user',
                'content': prompt_template
            },
            {
                'role': 'assistant',
                'content': response_template
            },
            {
                'role': 'user',
                'content': f'Do the same for the following snippet: `{selected_snippet}`'  # noqa: E501
            }
        ]
    )

image_prompt = completion.choices[0].message.content

print(selected_snippet)
print(image_prompt)

Some Republicans Were Willing to Compromise on Abortion Ban Exceptions. Activists Made Sure They Didn’t.

Imaginary Character: Harmony Stridefall

Description:
Harmony Stridefall, a fierce and enigmatic warrior, is of the elven race, known for their ethereal beauty and grace. With iridescent purple skin, opalescent eyes, and flowing hair shimmering like moonlit silver, they emanate an aura of profound strength and determination. Clad in intricately designed armor symbolizing balance, they bear the scars of battles fought for justice.

Backstory:
Harmony Stridefall was born within an ancient order of warriors dedicated to protecting the sacred balance of life. Imbued with the ancient powers of the forest, they discovered a hidden chamber containing a mystical amulet. This amulet granted them heightened abilities and a resolute commitment to safeguarding the rights of all living beings from unjust legislation.

Main Superpower:
Nature's Redemption - Harmony Stridefall possesses the abili

In [37]:
prompt = image_prompt.replace('\n', '').split('Description:')[-1].split('Backstory:')[0]
post_text = image_prompt.replace('\n', '').split('Backstory:')[-1]

In [38]:
url = "https://cloud.leonardo.ai/api/rest/v1/generations"

prompt = prompt

#### Call Leonardo AI

In [49]:

url = "https://cloud.leonardo.ai/api/rest/v1/generations"

payload = {
    "height": 512,
    # alch leonardov2
    "modelId": "1e60896f-3c26-4296-8ecc-53e2afecc132",
    # rpgv5
    # "modelId": "f1929ea3-b169-4c18-a16c-5d58b4292c69",
    "prompt": prompt,
    "width": 512,
    "alchemy": True,
    "highResolution": True,
    "nsfw": True,
    "num_images": 1,
    "promptMagic": True,
    #"photoReal": True,
    #"photoRealStrength": 0.5,
    "presetStyle": "CINEMATIC",
    "expandedDomain": True
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)
for k in range(20):
    print(f'Sleeping...{k}')
    time.sleep(1)

{"sdGenerationJob":{"generationId":"ed5d4e62-cd97-46c8-8ef3-e7e128ef4ae2","apiCreditCost":24}}
Sleeping...0
Sleeping...1
Sleeping...2
Sleeping...3
Sleeping...4
Sleeping...5
Sleeping...6
Sleeping...7
Sleeping...8
Sleeping...9
Sleeping...10
Sleeping...11
Sleeping...12
Sleeping...13
Sleeping...14
Sleeping...15
Sleeping...16
Sleeping...17
Sleeping...18
Sleeping...19


In [50]:
response.json()

{'sdGenerationJob': {'generationId': 'ed5d4e62-cd97-46c8-8ef3-e7e128ef4ae2',
  'apiCreditCost': 24}}

In [51]:
gen_id = response.json()['sdGenerationJob']['generationId']

#### Grab the generated image from a CDN

In [52]:
url = f"https://cloud.leonardo.ai/api/rest/v1/generations/{gen_id}"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"generations_by_pk":{"generated_images":[{"url":"https://cdn.leonardo.ai/users/8c647f87-f517-478a-95a7-e7d8f2dc1271/generations/ed5d4e62-cd97-46c8-8ef3-e7e128ef4ae2/Leonardo_Diffusion_XL_Harmony_Stridefall_a_fierce_and_enigmati_0.jpg","nsfw":false,"id":"d4b3dfdd-fd7f-43e1-a884-1b0e86646930","likeCount":0,"generated_image_variation_generics":[]}],"modelId":"1e60896f-3c26-4296-8ecc-53e2afecc132","prompt":"Harmony Stridefall, a fierce and enigmatic warrior, is of the elven race, known for their ethereal beauty and grace. With iridescent purple skin, opalescent eyes, and flowing hair shimmering like moonlit silver, they emanate an aura of profound strength and determination. Clad in intricately designed armor symbolizing balance, they bear the scars of battles fought for justice.","negativePrompt":"","imageHeight":512,"imageWidth":512,"inferenceSteps":30,"seed":741720576,"public":false,"scheduler":"EULER_DISCRETE","sdVersion":"SDXL_0_9","status":"COMPLETE","presetStyle":"CINEMATIC","initS

In [53]:
print(post_text)

Harmony Stridefall was born within an ancient order of warriors dedicated to protecting the sacred balance of life. Imbued with the ancient powers of the forest, they discovered a hidden chamber containing a mystical amulet. This amulet granted them heightened abilities and a resolute commitment to safeguarding the rights of all living beings from unjust legislation.Main Superpower:Nature's Redemption - Harmony Stridefall possesses the ability to commune with nature, channeling its elemental forces in defense of fundamental rights. They can summon powerful gusts of wind to shield those in danger, heal wounds with the soothing essence of plants, and create impenetrable barriers woven from the strength of ancient trees. In the face of any threat to reproductive rights, they empower activists to rise and combat injustice with unwavering determination, ensuring no compromise in the face of oppression.
